In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyngrok
!pip install flask-ngrok

In [ ]:
!ngrok authtoken 25cEjT1YXDcaCHC8aH56oPkjY2z_3XgezFTFRXKa2QME15ztn

In [ ]:
!pip install -U sentence-transformers

In [3]:
import pandas as pd 
import numpy as np
import re
import pickle
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


import flask
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok

In [4]:
def text_preprocessing(text):


  #Decontractions, replace words like below to full words. 
  text = re.sub(r"n\'t", " not", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'s", " is", text)
  text = re.sub(r"\'d", " would", text)
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r"\'t", " not", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"\'m", " am", text)

  #Removing tags like "< anyword >"
  #text = re.sub('\<.*?\>',' ',text)
  #Removing all the newlines('\n'), tabs('\t'), "-"
  #text = re.sub('\n|\t|\r|-',' ',text)
  #Removing all the data which are present in the brackets
  text = re.sub('\(.*?\)',' ',text)
  #Removing all the data which are present in the flower brackets
  text = re.sub('\{.*?\}',' ',text)
  #Removing URL's
  text = re.sub(r'http\S+', ' ', text)
  #Removing special characters
  text = re.sub('[?|!|\'|"|#]',' ', text)
  text = re.sub('[.|,|)|(|\|/]',' ', text)
  #Replacing numbers with word counterpart
  #https://stackoverflow.com/questions/22545114/python-regex-sub-use-match-as-dict-key-in-substitution
  num =  {'1':'one', '2':'two', '3':'three', '4':'four', '5':'five', '6':'six', '7':'seven', '8':'eight', '9':'nine', '0':'zero'}
  text = re.sub('[0-9]', lambda m: num.get(m.group()), text)
  #Removing non alphabets
  text = re.sub("[^a-zA-Z]",' ', text)
  #Converting text to lower case
  text = text.lower()
  text = ' '.join(text.split())




  return text

In [5]:
#Function to calculate sentence embeddings of question title
def Sentence_Embeddings(data, model):
  embeddings = []
  for i in tqdm(data):
    embeddings.append(model.encode(i))
  return embeddings


#Function to calculate cosine similarity
def cos_sim(a,b):
  sim = cosine_similarity(a,b)
  sim = [item for sublist in sim for item in sublist]
  #top5 = (np.argsort(sim)[-5:])
  return(sim)



#function to predict similar questions
def prediction(title_embeddings, model, question, df):
  question = text_preprocessing(question)
  question_embedd = Sentence_Embeddings([question], model)
  similarity=[]
  similarity.extend(cos_sim(title_embeddings[:400000], question_embedd))
  similarity.extend(cos_sim(title_embeddings[400000:800000], question_embedd))
  similarity.extend(cos_sim(title_embeddings[800000:], question_embedd))
  #similarity=cos_sim(title_embeddings[:], question_embedd)
  #return similarity
  top5 = (np.argsort(similarity)[-5:][::-1])
  return(df['Title'].iloc[top5].values)

In [6]:
# Running the flask app
app = Flask(__name__, template_folder="/content/drive/MyDrive/template", static_folder="/content/drive/MyDrive/static") #passing path of html and static files
run_with_ngrok(app)
#load model using pickle
model = pickle.load(open("/content/drive/MyDrive/mpnet_base.pkl", "rb"))
embeddings = pickle.load(open("/content/drive/MyDrive/BERT_title_embeddings.pkl", "rb"))
df = pd.read_csv("/content/drive/MyDrive/questions_title.csv")

@app.route('/', methods=['GET'])
def home():
    return render_template('index.html')

@app.route('/', methods=['POST'])
def predict():
    
    question = request.form.to_dict()
    print(question)
    title = question['Title']
    
    #question = str(question)
    #print(question)
    pred = prediction(embeddings, model, title, df)

    return render_template('index.html', prediction_text1='Similar Questions ', prediction_text2='1- {}'.format(pred[0]), prediction_text3='2-{}'.format(pred[1]), prediction_text4='3- {}'.format(pred[2]), prediction_text5='4- {}'.format(pred[3]), prediction_text6='5- {}'.format(pred[4]))



In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://88ef-35-185-71-83.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [28/Feb/2022 12:09:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2022 12:09:33] "GET / HTTP/1.1" 200 -


{'Title': 'combine 2 sentences in java'}


100%|██████████| 1/1 [00:00<00:00,  2.55it/s]
127.0.0.1 - - [28/Feb/2022 12:10:00] "POST / HTTP/1.1" 200 -


{'Title': 'Combine 2 sentences in java'}


100%|██████████| 1/1 [00:00<00:00,  7.93it/s]
127.0.0.1 - - [28/Feb/2022 12:10:47] "POST / HTTP/1.1" 200 -
